# set up

In [3]:
# clone yolov5 repo
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt # install dependencies
%pip install -q roboflow
%pip install loguru

Cloning into 'yolov5'...
remote: Enumerating objects: 15845, done.
remote: Counting objects: 100% (77/77), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 15845 (delta 31), reused 39 (delta 13), pack-reused 15768
Receiving objects: 100% (15845/15845), 14.66 MiB | 20.99 MiB/s, done.
Resolving deltas: 100% (10843/10843), done.
/content/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 605.6/605.6 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.8 MB/s eta 0:00:00


In [1]:
from roboflow import Roboflow
rf = Roboflow(api_key="gMTVjr2tVu8feJmkrnPs")
project = rf.workspace("aps360-project-6tmc3").project("test-set-r5re0")
dataset = project.version(1).download("yolov5")

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to Test-Set-1 in yolov5pytorch:: 100%|██████████| 966/966 [00:00<00:00, 1860.81it/s]


In [34]:
from utils.torch_utils import select_device, smart_inference_mode
from models.yolo import Model
from utils.general import (LOGGER, TQDM_BAR_FORMAT, check_amp, check_dataset, check_file, check_git_info,
                           check_git_status, check_img_size, check_requirements, check_suffix, check_yaml, colorstr,
                           get_latest_run, increment_path, init_seeds, intersect_dicts, labels_to_class_weights,
                           labels_to_image_weights, methods, one_cycle, print_args, print_mutation, strip_optimizer,
                           yaml_save)
from utils.torch_utils import (EarlyStopping, ModelEMA, de_parallel, select_device, smart_DDP, smart_optimizer,
                               smart_resume, torch_distributed_zero_first)

from utils.loss import ComputeLoss
import torch
import os
import yaml
LOCAL_RANK = int(os.getenv('LOCAL_RANK', -1))  # https://pytorch.org/docs/stable/elastic/run.html
RANK = int(os.getenv('RANK', -1))
WORLD_SIZE = int(os.getenv('WORLD_SIZE', 1))
import numpy as np
import matplotlib.pyplot as plt
from utils.dataloaders import create_dataloader
from tqdm import tqdm
from loguru import logger
from utils.metrics import bbox_iou


In [5]:
def _load_model(weights_path, device, nc, anchors):
  '''
    @param data_folder: path to the data.yaml file
    return a model that is loaded with the specified weights
  '''
  ckpt = torch.load(weights_path, map_location="cpu")
  model = Model(ckpt['model'].yaml, ch=3, nc=nc, anchors=anchors).to(device)  # create
  exclude = []
  # exclude = ['anchor'] if anchors else []
  csd = ckpt['model'].float().state_dict()  # checkpoint state_dict as FP32
  csd = intersect_dicts(csd, model.state_dict(), exclude=exclude)  # intersect
  model.load_state_dict(csd, strict=False)  # load
  print(f'Transferred {len(csd)}/{len(model.state_dict())} items from {weights_path}')  # report)
  return model

In [37]:
class ComputeIndividualLoss(ComputeLoss):
  def __init__(self, model, autobalance=False):
    super().__init__(model, autobalance)

  # override the __call__ function to return the component loss tensors
  def __call__(self, p, targets):  # predictions, targets
    lcls = torch.zeros(1, device=self.device)  # class loss
    lbox = torch.zeros(1, device=self.device)  # box loss
    lobj = torch.zeros(1, device=self.device)  # object loss
    tcls, tbox, indices, anchors = self.build_targets(p, targets)  # targets

    # Losses
    for i, pi in enumerate(p):  # layer index, layer predictions
        b, a, gj, gi = indices[i]  # image, anchor, gridy, gridx
        tobj = torch.zeros(pi.shape[:4], dtype=pi.dtype, device=self.device)  # target obj

        n = b.shape[0]  # number of targets
        if n:
            # pxy, pwh, _, pcls = pi[b, a, gj, gi].tensor_split((2, 4, 5), dim=1)  # faster, requires torch 1.8.0
            pxy, pwh, _, pcls = pi[b, a, gj, gi].split((2, 2, 1, self.nc), 1)  # target-subset of predictions

            # Regression
            pxy = pxy.sigmoid() * 2 - 0.5
            pwh = (pwh.sigmoid() * 2) ** 2 * anchors[i]
            pbox = torch.cat((pxy, pwh), 1)  # predicted box
            iou = bbox_iou(pbox, tbox[i], CIoU=True).squeeze()  # iou(prediction, target)
            lbox += (1.0 - iou).mean()  # iou loss

            # Objectness
            iou = iou.detach().clamp(0).type(tobj.dtype)
            if self.sort_obj_iou:
              j = iou.argsort()
              b, a, gj, gi, iou = b[j], a[j], gj[j], gi[j], iou[j]
            if self.gr < 1:
                iou = (1.0 - self.gr) + self.gr * iou
            tobj[b, a, gj, gi] = iou  # iou ratio

            # Classification
            if self.nc > 1:  # cls loss (only if multiple classes)
                t = torch.full_like(pcls, self.cn, device=self.device)  # targets
                t[range(n), tcls[i]] = self.cp
                lcls += self.BCEcls(pcls, t)  # BCE
            # Append targets to text file
            # with open('targets.txt', 'a') as file:
            #     [file.write('%11.5g ' * 4 % tuple(x) + '\n') for x in torch.cat((txy[i], twh[i]), 1)]

        obji = self.BCEobj(pi[..., 4], tobj)
        lobj += obji * self.balance[i]  # obj loss
        if self.autobalance:
            self.balance[i] = self.balance[i] * 0.9999 + 0.0001 / obji.detach().item()

    if self.autobalance:
        self.balance = [x / self.balance[self.ssi] for x in self.balance]
    lbox *= self.hyp['box']
    lobj *= self.hyp['obj']
    lcls *= self.hyp['cls']
    bs = tobj.shape[0]  # batch size

    return (lbox + lobj + lcls), torch.cat((lbox, lobj, lcls)).detach(), lbox, lobj, lcls

In [38]:
def evaluate(weights_path,
              data_folder,
              hyperparam_path,
              batch_size=32,
              imgsz=640,
              device='',
              seed=0,
              workers=2,
              label_smoothing=0.0):

  single_cls = False
  # load in the hyperparameters
  if isinstance(hyperparam_path, str):
        with open(hyperparam_path, errors='ignore') as f:
            hyp = yaml.safe_load(f)  # load hyps dict

  cuda = device.type != 'cpu'
  init_seeds(seed + 1 + RANK, deterministic=True)

  # load data dict
  with torch_distributed_zero_first(LOCAL_RANK):
      data_dict = check_dataset(data_folder)  # check if None
  nc = int(data_dict["nc"])
  names = data_dict["names"]
  is_coco = False

  # load model with weights
  model = _load_model(weights_path, device, nc, hyp.get("anchors"))
  amp = check_amp(model)  # check AMP

  gs = max(int(model.stride.max()), 32)  # grid size (max stride)
  imgsz = check_img_size(imgsz, gs, floor=gs * 2)  # verify imgsz is gs-multiple

  ema = ModelEMA(model) if RANK in {-1, 0} else None

  test_path = data_dict['test']
  test_loader, dataset = create_dataloader(test_path,
                                            imgsz,
                                            batch_size // WORLD_SIZE,
                                            gs,
                                            single_cls,
                                            hyp=hyp,
                                            augment=False,
                                            cache=None,
                                            rect=False, #unless you specify this flag when training
                                            rank=LOCAL_RANK,
                                            workers=workers,
                                            image_weights=False, # unless you specify this flag when training
                                            quad=False, # unless you specify this flag when training
                                            prefix=colorstr('test: '),
                                            shuffle=True,
                                            seed=seed)
  labels = np.concatenate(dataset.labels, 0)
  mlc = int(labels[:, 0].max())  # max label class
  assert mlc < nc, f'Label class {mlc} exceeds nc={nc} in {data_folder}. Possible class labels are 0-{nc - 1}'

  # Model attributes
  nl = de_parallel(model).model[-1].nl  # number of detection layers (to scale hyps)
  hyp['box'] *= 3 / nl  # scale to layers
  hyp['cls'] *= nc / 80 * 3 / nl  # scale to classes and layers
  hyp['obj'] *= (imgsz / 640) ** 2 * 3 / nl  # scale to image size and layers
  hyp['label_smoothing'] = label_smoothing
  model.nc = nc  # attach number of classes to model
  model.hyp = hyp  # attach hyperparameters to model
  model.class_weights = labels_to_class_weights(dataset.labels, nc).to(device) * nc  # attach class weights
  model.names = names

  model.train()
  compute_loss = ComputeIndividualLoss(model)
  pbar = enumerate(test_loader)
  pbar = tqdm(pbar, total=len(test_loader), bar_format=TQDM_BAR_FORMAT)  # progress bar

  for batch_i, (im, targets, paths, shapes) in (pbar):
      if cuda:
          im = im.to(device, non_blocking=True).float() / 255
          targets = targets.to(device)
      nb, _, height, width = im.shape  # batch size, channels, height, width
      with torch.no_grad():
          pred = model(im)

          compute_loss(pred, targets)

In [39]:
evaluate("/content/bestC.pt", "/content/yolov5/Test-Set-1/data.yaml", 'data/hyps/hyp.scratch-low.yaml', device=torch.device("cuda:0"))


                 from  n    params  module                                  arguments                     
  0                -1  1      5280  models.common.Conv                      [3, 48, 6, 2, 2]              
  1                -1  1     41664  models.common.Conv                      [48, 96, 3, 2]                
  2                -1  2     65280  models.common.C3                        [96, 96, 2]                   
  3                -1  1    166272  models.common.Conv                      [96, 192, 3, 2]               
  4                -1  4    444672  models.common.C3                        [192, 192, 4]                 
  5                -1  1    664320  models.common.Conv                      [192, 384, 3, 2]              
  6                -1  6   2512896  models.common.C3                        [384, 384, 6]                 
  7                -1  1   2655744  models.common.Conv                      [384, 768, 3, 2]              
  8                -1  2   4134912  

Transferred 481/481 items from /content/bestC.pt


test: Scanning /content/yolov5/Test-Set-1/test/labels.cache... 480 images, 1 backgrounds, 0 corrupt: 100%|██████████| 480/480 [00:00<?, ?it/s]
  0%|          | 0/15 [00:00<?, ?it/s]2023-08-01 05:07:04.576 | INFO     | __main__:__call__:11 - len of 3
2023-08-01 05:07:04.588 | INFO     | __main__:__call__:44 - t shape torch.Size([29, 2])
2023-08-01 05:07:04.593 | INFO     | __main__:__call__:45 - pcls shape torch.Size([29, 2])
2023-08-01 05:07:04.600 | INFO     | __main__:__call__:46 - t tensor([[1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
     

AssertionError: ignored

# baseline model

In [11]:
import torch.nn as nn
import torch.nn.functional as F

In [12]:
class BaselineNet(nn.Module):
    def __init__(self):
        super(BaselineNet, self).__init__()
        self.name = "baseline"
        self.conv = nn.Conv2d(3, 5, 5, stride = 2)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(4500, 5 * 7 * 7)
        self.fc2 = nn.Linear(5 * 7 * 7, 1)

    def forward(self, x):
        x = self.pool(F.relu(self.conv(x)))
        x = self.pool(x)
        x = x.view(-1, 4500)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = x.squeeze(1) # Flatten to [batch_size]
        return x

In [ ]:
baseline_model = BaselineNet()
